In [11]:
# Import necessary libraries
import numpy as np  # For numerical operations
import os  # For interacting with the operating system
import cv2  # For image processing
from sklearn.model_selection import train_test_split  # For splitting the dataset into training and testing sets
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # For evaluating the model
from sklearn.svm import SVC  # For Support Vector Machine classifier
from sklearn.neural_network import MLPClassifier  # For Neural Network classifier
from skimage.feature import hog  # For extracting Histogram of Oriented Gradients (HOG) features from images
import tensorflow as tf  # For deep learning operations
from tensorflow.keras.models import Sequential  # For creating a linear stack of layers for the neural network
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization  # For defining different types of layers in the neural network
from tensorflow.keras.optimizers import Adam, SGD  # For optimizing the neural network

In [ ]:
# Code to extract features from images

def extract_features(image_path, use_hog):
    """Load image and extract features (HOG or raw pixels)."""
    # Load the image in grayscale mode
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    
    # Check if the image was loaded successfully
    if image is None:
        raise ValueError(f"Image not found or unable to load: {image_path}")

    # Resize the image to a fixed size (128x128)
    image = cv2.resize(image, (128, 128))

    if use_hog:
        # Extract HOG features for ML classifiers
        features, _ = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
        return features  # Return 1D feature vector
    else:
        # Normalize pixel values for CNN (range 0-1)
        return image.reshape(128, 128, 1) / 255.0  # Return 2D image for CNN


Part A:
Binary Classification Using Handcrafted Features and ML Classifiers (4 Marks) 
- i. Extract handcrafted features from the dataset. 
- ii. Train and evaluate at least two machine learning classifiers (e.g., SVM, Neural 
network) to classify faces as "with mask" or "without mask." 
- iii. Report and compare the accuracy of the classifiers. 

In [5]:
# load the dataset for normal ML models require 1D feature vector (HOG)

# Set dataset path
dataset_path = "dataset"

# Storage for dataset
data = []
labels = []
label_counts = {'with_mask': 0, 'without_mask': 0}


# Load dataset
for label in ['with_mask', 'without_mask']:
    folder_path = os.path.join(dataset_path, label)
    # print(f"Processing folder: {folder_path}")
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        # print(f"Processing file: {file_path}")
        try:
            features = extract_features(file_path, use_hog=True)
            data.append(features)
            labels.append(1 if label == 'with_mask' else 0)
            label_counts[label] += 1
        except ValueError as e:
            print(e) # Print the error message
        except Exception as e:
            print(f"Unexpected error processing file {file_path}: {e}")


# Convert to numpy arrays
X = np.array(data)
y = np.array(labels)

# Print dataset shape
print(f"Feature array shape: {X.shape}, Labels shape: {y.shape}")
print(f"Number of 'with_mask' images: {label_counts['with_mask']}")
print(f"Number of 'without_mask' images: {label_counts['without_mask']}")

Feature array shape: (4095, 8100), Labels shape: (4095,)
Number of 'with_mask' images: 2165
Number of 'without_mask' images: 1930


In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Train and evaluate SVM classifier
svm_clf = SVC()
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)

# Report the accuracy of the SVM classifier
print(f"SVM Accuracy: {svm_accuracy}")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_svm))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_svm))

SVM Accuracy: 0.9304029304029304
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.89      0.92       366
           1       0.91      0.96      0.94       453

    accuracy                           0.93       819
   macro avg       0.93      0.93      0.93       819
weighted avg       0.93      0.93      0.93       819

Confusion Matrix:
[[325  41]
 [ 16 437]]


In [8]:
# Train and evaluate Neural Network classifier
nn_clf = MLPClassifier()
nn_clf.fit(X_train, y_train)
y_pred_nn = nn_clf.predict(X_test)
nn_accuracy = accuracy_score(y_test, y_pred_nn)

# Report the accuracy of the MLP classifier
print(f"Neural Network Accuracy: {nn_accuracy}")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_nn))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_nn))

Neural Network Accuracy: 0.894993894993895
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       366
           1       0.90      0.91      0.91       453

    accuracy                           0.89       819
   macro avg       0.89      0.89      0.89       819
weighted avg       0.89      0.89      0.89       819

Confusion Matrix:
[[319  47]
 [ 39 414]]


In [14]:
# Load the dataset for CNN model require 2D image array
# Set dataset path
dataset_path = "dataset"

# Storage for dataset
data = []
labels = []
label_counts = {'with_mask': 0, 'without_mask': 0}


# Load dataset
for label in ['with_mask', 'without_mask']:
    folder_path = os.path.join(dataset_path, label)
    # print(f"Processing folder: {folder_path}")
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        # print(f"Processing file: {file_path}")
        try:
            features = extract_features(file_path, use_hog=False)
            data.append(features)
            labels.append(1 if label == 'with_mask' else 0)
            label_counts[label] += 1
        except ValueError as e:
            print(e) # Print the error message
        except Exception as e:
            print(f"Unexpected error processing file {file_path}: {e}")


# Convert to numpy arrays
X = np.array(data)
y = np.array(labels)

# Print dataset shape
print(f"Feature array shape: {X.shape}, Labels shape: {y.shape}")
print(f"Number of 'with_mask' images: {label_counts['with_mask']}")
print(f"Number of 'without_mask' images: {label_counts['without_mask']}")



Feature array shape: (4095, 128, 128, 1), Labels shape: (4095,)
Number of 'with_mask' images: 2165
Number of 'without_mask' images: 1930


In [15]:
# One-hot encode labels for categorical cross-entropy loss
y = tf.keras.utils.to_categorical(y, 2)

# Split dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes to verify
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")


Training set shape: (3276, 128, 128, 1), (3276, 2)
Testing set shape: (819, 128, 128, 1), (819, 2)


In [ ]:
# -------------------------- CNN Model 1 --------------------------
def create_cnn_model_1():
    """CNN Model 1: Uses Adam optimizer, deeper layers, and dropout."""
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Prevent overfitting
        Dense(2, activation='softmax')  # 2 output classes
    ])

    # Compile model using Adam optimizer
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    
    return model

# Train CNN Model 1
cnn_model_1 = create_cnn_model_1()
print("\nTraining CNN Model 1 (Adam, deeper layers, higher dropout)...")
history_1 = cnn_model_1.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
test_loss_1, test_acc_1 = cnn_model_1.evaluate(X_test, y_test)
print(f" CNN Model 1 Test Accuracy: {test_acc_1:.4f}")



🔹 Training CNN Model 1 (Adam, deeper layers, higher dropout)...
Epoch 1/5
103/103 [==============================] - 86s 830ms/step - loss: 0.5501 - accuracy: 0.7085 - val_loss: 0.3828 - val_accuracy: 0.8278
Epoch 2/5
103/103 [==============================] - 72s 700ms/step - loss: 0.3390 - accuracy: 0.8538 - val_loss: 0.2726 - val_accuracy: 0.8864
Epoch 3/5
103/103 [==============================] - 71s 686ms/step - loss: 0.2387 - accuracy: 0.9011 - val_loss: 0.2213 - val_accuracy: 0.9170
Epoch 4/5
103/103 [==============================] - 75s 730ms/step - loss: 0.1837 - accuracy: 0.9280 - val_loss: 0.1996 - val_accuracy: 0.9145
Epoch 5/5
26/26 [==============================] - 5s 207ms/step - loss: 0.1943 - accuracy: 0.9328
 CNN Model 1 Test Accuracy: 0.9328


In [21]:
# Predict the labels for the test set
y_pred_cnn_1 = cnn_model_1.predict(X_test)
y_pred_cnn_1_classes = np.argmax(y_pred_cnn_1, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Generate the confusion matrix
conf_matrix_cnn_1 = confusion_matrix(y_test_classes, y_pred_cnn_1_classes)

# Print the confusion matrix
print("Confusion Matrix for CNN Model 1:")
print(conf_matrix_cnn_1)

26/26 [==============================] - 3s 124ms/step
Confusion Matrix for CNN Model 1:
[[350  16]
 [ 39 414]]


In [ ]:
# -------------------------- CNN Model 2 --------------------------
def create_cnn_model_2():
    """CNN Model 2: Uses SGD optimizer, batch normalization, and fewer layers."""
    model = Sequential([
        Conv2D(32, (5, 5), activation='relu', input_shape=(128, 128, 1)),
        BatchNormalization(),  # Normalize activations
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (5, 5), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),  # Lower dropout than model 1
        Dense(2, activation='softmax')  # 2 output classes
    ])

    # Compile model using SGD optimizer (lower learning rate)
    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(learning_rate=0.005, momentum=0.9),
                  metrics=['accuracy'])
    
    return model


# Train CNN Model 2
cnn_model_2 = create_cnn_model_2()
print("\nTraining CNN Model 2 (SGD, batch norm, fewer layers)...")
history_2 = cnn_model_2.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
test_loss_2, test_acc_2 = cnn_model_2.evaluate(X_test, y_test)
print(f" CNN Model 2 Test Accuracy: {test_acc_2:.4f}")



🔹 Training CNN Model 2 (SGD, batch norm, fewer layers)...
Epoch 1/5
103/103 [==============================] - 105s 1s/step - loss: 0.7713 - accuracy: 0.7314 - val_loss: 0.9722 - val_accuracy: 0.5739
Epoch 2/5
103/103 [==============================] - 116s 1s/step - loss: 0.4182 - accuracy: 0.8071 - val_loss: 0.7833 - val_accuracy: 0.7106
Epoch 3/5
103/103 [==============================] - 103s 1s/step - loss: 0.3260 - accuracy: 0.8544 - val_loss: 2.4855 - val_accuracy: 0.6349
Epoch 4/5
103/103 [==============================] - 100s 970ms/step - loss: 0.2848 - accuracy: 0.8788 - val_loss: 0.9367 - val_accuracy: 0.7582
Epoch 5/5
26/26 [==============================] - 9s 333ms/step - loss: 0.5936 - accuracy: 0.7155
 CNN Model 2 Test Accuracy: 0.7155


In [22]:
# Predict the labels for the test set using CNN Model 2
y_pred_cnn_2 = cnn_model_2.predict(X_test)
y_pred_cnn_2_classes = np.argmax(y_pred_cnn_2, axis=1)

# Generate the confusion matrix
conf_matrix_cnn_2 = confusion_matrix(y_test_classes, y_pred_cnn_2_classes)

# Print the confusion matrix
print("Confusion Matrix for CNN Model 2:")
print(conf_matrix_cnn_2)

26/26 [==============================] - 5s 182ms/step
Confusion Matrix for CNN Model 2:
[[346  20]
 [213 240]]


### Accuracy Comparison

In [20]:
# Compare model performance
print("\nComparison of CNN Models:")
print(f"Model 1 SVC: {svm_accuracy:.4f}")
print(f"Model 2 MLP neural network: {nn_accuracy:.4f}")
print(f"Model 1 (Adam, deeper layers) Accuracy: {test_acc_1:.4f}")
print(f"Model 2 (SGD, batch norm) Accuracy: {test_acc_2:.4f}")


Comparison of CNN Models:
Model 1 SVC: 0.9304
Model 2 MLP neural network: 0.8950
Model 1 (Adam, deeper layers) Accuracy: 0.9328
Model 2 (SGD, batch norm) Accuracy: 0.7155
